# TF高级API - eager模式

更多高级API参考
https://github.com/GoogleCloudPlatform/tf-estimator-tutorials

In [1]:
# -*- coding: utf-8 -*-

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from six.moves import xrange

开启Eager模式

In [2]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe

# set Eager API
# use Eager model so that without tf.Session()
tfe.enable_eager_execution()

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


numpy与tensorflow相互转换

In [3]:
# numpy compatibility
import numpy as np

tf_tensor = tf.add(np.zeros(shape=(1, 2)), 1)
print(type(tf_tensor), '=', tf_tensor)

np_tensor = tf_tensor.numpy()
print(type(np_tensor), '=', np_tensor)

<class 'EagerTensor'> = tf.Tensor([[1. 1.]], shape=(1, 2), dtype=float64)
<class 'numpy.ndarray'> = [[1. 1.]]


Eager模式下用于检测，张良位置

In [4]:
with tf.device('CPU:0'):
    tensor = tf.random_uniform(shape=(1, 2))
    print('tensor is on [GPU:0]?', tensor.device.endswith('GPU:0'))


tensor is on [GPU:0]? False


Eager模式下，模型的创建

In [5]:
from tensorflow.examples.tutorials.mnist import input_data


class MNIST():
    def __init__(self, batch_size=1000):
        mnist = input_data.read_data_sets('../MNIST_data', one_hot=False)
        self.dataset = {
            'train': tf.data.Dataset.from_tensor_slices(
                (mnist.train.images, mnist.train.labels)).batch(batch_size),
            'test': tf.data.Dataset.from_tensor_slices(
                (mnist.test.images, mnist.test.labels)).batch(batch_size)}
        self.length = {'train': len(mnist.train.images) // batch_size,
                       'test': len(mnist.test.images) // batch_size}
        del mnist
        self.dataset_iter = {key: self.init_dataset(value) for key, value in self.dataset.items()}

    def init_dataset(self, dataset):
        return tfe.Iterator(dataset)

    def next(self, train=True):
        tag = 'train' if train else 'test'
        try:
            batch_images, batch_labels = self.dataset_iter[tag].next()
        except:
            self.dataset_iter[tag] = self.init_dataset(self.dataset[tag])
            batch_images, batch_labels = self.dataset_iter[tag].next()
        batch_labels = tf.cast(batch_labels, dtype=tf.int64)
        return batch_images, batch_labels

    def __len__(self, train=True):
        tag = 'train' if train else 'test'
        return self.length[tag]


class LinearNet(tfe.Network):
    def __init__(self):
        super(LinearNet, self).__init__()
        self.layer1 = self.track_layer(tf.layers.Dense(128, activation=tf.nn.relu))
        self.layer2 = self.track_layer(tf.layers.Dense(classes_num))

        self.optimizer = tf.train.AdamOptimizer(learning_rate=1e-3)
        self.grad = tfe.implicit_gradients(self.loss_fn)

    def call(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        return x

    def loss_fn(self, inputs, labels):
        return tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=self.call(inputs), labels=labels))

    def accuracy_fn(self, inputs, labels):
        predictions = tf.argmax(tf.nn.softmax(self.call(inputs)), axis=-1)
        prob = tf.equal(labels, predictions)
        return tf.reduce_mean(tf.cast(prob, dtype=tf.float32))

    def optimize(self, inputs, labels):
        gradients = self.grad(inputs, labels)
        self.optimizer.apply_gradients(gradients)


batch_size = 500
mnist = MNIST()
classes_num = 10
linearNet = LinearNet()

training_episode = 5
for episode in xrange(training_episode):
    loss, accuracy = 0.0, 0.0
    for i in xrange(len(mnist)):
        batch_images, batch_labels = mnist.next()
        loss += linearNet.loss_fn(batch_images, batch_labels)
        accuracy += linearNet.accuracy_fn(batch_images, batch_labels)
        linearNet.optimize(batch_images, batch_labels)
    loss /= len(mnist)
    accuracy /= len(mnist)
    print('Episode %s / %s, loss:%-.5f, acc:%-.5f' % (episode, training_episode, loss, accuracy))

loss, accuracy = 0.0, 0.0
for i in xrange(mnist.__len__(train=False)):
    batch_images, batch_labels = mnist.next()
    loss += linearNet.loss_fn(batch_images, batch_labels)
    accuracy += linearNet.accuracy_fn(batch_images, batch_labels)
    linearNet.optimize(batch_images, batch_labels)
loss /= mnist.__len__(train=False)
accuracy /= mnist.__len__(train=False)
print('Evaluation, loss:%-.5f, acc:%-.5f' % (loss, accuracy))


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Episode 0 / 5, loss:0.83987, acc:0.78196
Episode 1 / 5, loss:0.33577, acc:0.90691
Episode 2 / 5, loss:0.27109, acc:0.92502
Episode 3 / 5, loss:0.23340, acc:0.93587
Episode 4 / 5, loss:0.20591, acc:0.94331
Evaluation, loss:0.19729, acc:0.94330
